# Register cardio_train and quitline_services Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

In [1]:
import boto3
import sagemaker
import pandas as pd

# Initialize SageMaker session
sess = sagemaker.Session()
bucket = "cardiovale-solutions-datascience-pipeline"  # Set your S3 bucket name
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
ingest_create_athena_table_cardio_passed = False

# Import PyAthena

In [3]:
from pyathena import connect

# Create Athena Table from Local cardio_train-checkpoint.csv and Quitline_Services_Available_Medications.csv Files

In [4]:
!aws s3 ls --recursive $s3_private_path_tsv

2025-03-16 05:44:59 aws-athena-query-results-786782285170-us-east-1
2025-03-07 06:28:33 cardiovale-solutions-datascience-pipeline
2025-02-27 06:09:40 sagemaker-studio-gyyb8wtvkop
2025-02-27 06:09:42 sagemaker-us-east-1-786782285170


In [5]:
s3_private_path_cardio = "s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train/"
s3_private_path_quitline = "s3://cardiovale-solutions-datascience-pipeline/raw-data/quitline_services/"


In [6]:
!aws s3 ls $s3_private_path_tsv --recursive


2025-03-16 05:44:59 aws-athena-query-results-786782285170-us-east-1
2025-03-07 06:28:33 cardiovale-solutions-datascience-pipeline
2025-02-27 06:09:40 sagemaker-studio-gyyb8wtvkop
2025-02-27 06:09:42 sagemaker-us-east-1-786782285170


In [7]:
# Set S3 staging directory
s3_staging_dir = "s3://cardiovale-solutions-datascience-pipeline/athena/staging/"

In [9]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [10]:
# Define SQL statement to drop table
# statement = "DROP TABLE IF EXISTS cardiovale_db.cardio_train;"
statement = "DROP TABLE IF EXISTS cardiovale_db.quitline_services;"

pd.read_sql(statement, conn)

/tmp/ipykernel_2030/3221176216.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [11]:
!aws s3 ls s3://cardiovale-solutions-datascience-pipeline/raw-data/ --recursive


2025-03-18 04:09:39        108 raw-data/Unsaved/2025/03/17/484c9471-fd67-4d25-b636-e00e45c84100.csv
2025-03-18 04:09:39        531 raw-data/Unsaved/2025/03/17/484c9471-fd67-4d25-b636-e00e45c84100.csv.metadata
2025-03-18 04:57:53        815 raw-data/Unsaved/2025/03/17/d8b20f72-41c5-4f03-b6f1-a116f1eb09b2.csv
2025-03-18 04:57:53       2746 raw-data/Unsaved/2025/03/17/d8b20f72-41c5-4f03-b6f1-a116f1eb09b2.csv.metadata
2025-03-18 04:51:22        815 raw-data/Unsaved/2025/03/17/ee3da1b3-b25b-44a0-b812-781a71eb0162.csv
2025-03-18 04:51:22       2746 raw-data/Unsaved/2025/03/17/ee3da1b3-b25b-44a0-b812-781a71eb0162.csv.metadata
2025-03-18 02:43:32    2941524 raw-data/cardio_train/cardio_train.csv
2025-03-18 02:45:52    2941524 raw-data/cardio_train/cardio_train_fixed.csv
2025-03-18 02:45:53    6934258 raw-data/quitline_services/Quitline___Services_Available___Medications_-_2010_To_Present_20250306.csv


In [13]:
ls -lah /home/sagemaker-user/ads-508-team/data/


total 29M
drwxr-xr-x  3 sagemaker-user users  270 Mar 18 02:41 ./
drwxr-xr-x 11 sagemaker-user users  202 Mar 16 18:36 ../
drwxr-xr-x  2 sagemaker-user users  136 Mar 17 05:39 .ipynb_checkpoints/
-rw-r--r--  1 sagemaker-user users 6.7M Mar  7 06:39 Quitline___Services_Available___Medications_-_2010_To_Present_20250306.csv
-rw-r--r--  1 sagemaker-user users 2.9M Mar  7 06:39 cardio_train.csv
-rw-r--r--  1 sagemaker-user users 2.9M Mar 17 22:37 cardio_train_fixed.csv
-rw-r--r--  1 sagemaker-user users 2.9M Mar 17 05:39 processed_cardio_train.csv
-rw-r--r--  1 sagemaker-user users 6.7M Mar 17 05:39 processed_quitline_services.csv
-rw-r--r--  1 sagemaker-user users 6.7M Mar 17 05:32 quitline_services.csv


## Using panda to process the data before athena

In [90]:
import pandas as pd

# Load the dataset with semicolon separator
file_path = "/home/sagemaker-user/ads-508-team/data/cardio_train_fixed.csv"
df = pd.read_csv(file_path, sep=";")

# Save as a comma-separated CSV
fixed_file_path = "/home/sagemaker-user/ads-508-team/data/cardio_train_cleaned.csv"
df.to_csv(fixed_file_path, sep=",", index=False)

print("CSV successfully converted to comma-separated format!")


CSV successfully converted to comma-separated format!


In [107]:
import pandas as pd

# Load the dataset (already comma-separated)
file_path = "/home/sagemaker-user/ads-508-team/data/cardio_train_fixed.csv"
df = pd.read_csv(file_path, sep=",")

# Save without extra quotes
fixed_file_path = "/home/sagemaker-user/ads-508-team/data/cardio_train_cleaned.csv"
df.to_csv(fixed_file_path, sep=",", index=False, quoting=3)  # quoting=3 -> No extra quotes

print("Successfully cleaned and saved CSV without quotes!")


Successfully cleaned and saved CSV without quotes!


In [99]:
!aws s3 cp /home/sagemaker-user/ads-508-team/data/cardio_train_cleaned.csv s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train_cleaned/


upload: ../data/cardio_train_cleaned.csv to s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train_cleaned/cardio_train_cleaned.csv


In [100]:
# Define SQL statement to drop table
# statement = "DROP TABLE IF EXISTS cardiovale_db.cardio_train;"
statement = "DROP TABLE IF EXISTS cardiovale_db.cardio_train_cleaned;"

pd.read_sql(statement, conn)

/tmp/ipykernel_2030/2202486819.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [81]:
!aws s3 ls s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train/


2025-03-18 02:43:32    2941524 cardio_train.csv
2025-03-18 05:25:36    2941524 cardio_train_cleaned.csv
2025-03-18 02:45:52    2941524 cardio_train_fixed.csv


In [61]:
# Use the absolute path
file_path = "/home/sagemaker-user/ads-508-team/data/quitline_services.csv"

# Load the dataset
quitline_df = pd.read_csv(file_path)

# Check for multi-line text issues
quitline_df.sample(5)  # Random samples to look for line break issues


,Year,Date,Date_Ref,LocationAbbr,LocationDesc,TopicType,TopicDesc,MeasureDesc,Sub-Measure,Variable,...,Number_of_Weeks_Offered,Limit_Per_Year,Comments,GeoLocation,TopicTypeId,TopicId,MeasureID,SOURCE,SubMeasureID,DisplayOrder
13067,2011,3,Jul-Sep,MN,Minnesota,Quitline,Services Available,Medications,Nicotine Gum,All Eligible Callers,...,4.0,2 times per year,NaN,"(46.35564873600049, -94.79420050299967)",QUI,900QUI,912MED,"National Quitline Data Warehouse, Office on Sm...",QUT06,6
15847,2010,4,Oct-Dec,UT,Utah,Quitline,Services Available,Medications,Nicotine Inhaler,NaN,...,NaN,NaN,NaN,"(39.360700171000474, -111.58713063499971)",QUI,900QUI,912MED,"National Quitline Data Warehouse, Office on Sm...",QUT09,9
10628,2015,1,Jan-Mar,IA,Iowa,Quitline,Services Available,Medications,Nicotine Lozenge,All Eligible Callers,...,8.0,Not specified by state,NaN,"(42.46940091300047, -93.81649055599968)",QUI,900QUI,912MED,"National Quitline Data Warehouse, Office on Sm...",QUT07,7
17756,2011,3,Jul-Sep,GU,Guam,Quitline,Services Available,Medications,Nicotine Patch,Medicare,...,4.0,1 time per year,NaN,"(13.444304, 144.793731)",QUI,900QUI,912MED,"National Quitline Data Warehouse, Office on Sm...",QUT05,5
6334,2016,3,Jul-Sep,AZ,Arizona,Quitline,Services Available,Medications,Bupropion (Zyban®),NaN,...,NaN,NaN,NaN,"(34.865970280000454, -111.76381127699972)",QUI,900QUI,912MED,"National Quitline Data Warehouse, Office on Sm...",QUT10,10


In [20]:
quitline_df = quitline_df.astype(str).apply(lambda x: x.str.replace(r'\n', ' ', regex=True))
quitline_df.to_csv("/home/sagemaker-user/ads-508-team/data/quitline_fixed.csv", index=False, sep=",")
!aws s3 cp /home/sagemaker-user/ads-508-team/data/quitline_fixed.csv s3://cardiovale-solutions-datascience-pipeline/raw-data/quitline_services/


upload: ../data/quitline_fixed.csv to s3://cardiovale-solutions-datascience-pipeline/raw-data/quitline_services/quitline_fixed.csv


In [101]:
# Set Athena parameters
database_name = "cardiovale_db"
table_1_name = "cardio_train_cleaned"
table_2_name = "quitline_fixed"

In [102]:
# Update Athena table to use cleaned CSV
statement_cardio = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         id INT,
         age INT,
         gender INT,
         height INT,
         weight FLOAT,
         ap_hi INT,
         ap_lo INT,
         cholesterol INT,
         gluc INT,
         smoke INT,
         alco INT,
         active INT,
         cardio INT
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, "cardio_train_cleaned", "s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train_cleaned/"
)

print(statement_cardio)


CREATE EXTERNAL TABLE IF NOT EXISTS cardiovale_db.cardio_train_cleaned(
         id INT,
         age INT,
         gender INT,
         height INT,
         weight FLOAT,
         ap_hi INT,
         ap_lo INT,
         cholesterol INT,
         gluc INT,
         smoke INT,
         alco INT,
         active INT,
         cardio INT
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
LOCATION 's3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train_cleaned/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [24]:
# SQL statement to execute for quitline_services table
statement_quitline = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Year INT,
         Date INT,
         Date_Ref STRING,
         LocationAbbr STRING,
         LocationDesc STRING,
         TopicType STRING,
         TopicDesc STRING,
         MeasureDesc STRING,
         Sub_Measure STRING,
         Variable STRING,
         Offered_for_Free STRING,
         Offered_for_Free_Text STRING,
         Geographic_Requirements STRING,
         Geographic_Requirements_Text STRING,
         Age_Requirements STRING,
         Age_Requirements_Text STRING,
         Readiness_to_Quit_Requirements STRING,
         Readiness_to_Quit_Requirements_Text STRING,
         Counseling_Requirements STRING,
         Counseling_Requirements_Text STRING,
         Physician_Approval_Required STRING,
         Physician_Approval_Required_Text STRING,
         Medical_Restrictions STRING,
         Medical_Restrictions_Text STRING,
         Insurance_Requirements STRING,
         Insurance_Requirements_Text STRING,
         Other_Requirements STRING,
         Other_Requirements_Text STRING,
         Limited_Supply STRING,
         Limited_Supply_Text STRING,
         Unlimited_Weeks_Offered STRING,
         Number_of_Weeks_Offered FLOAT,
         Limit_Per_Year STRING,
         Comments STRING,
         GeoLocation STRING,
         TopicTypeId STRING,
         TopicId STRING,
         MeasureID STRING,
         SOURCE STRING,
         SubMeasureID STRING,
         DisplayOrder INT
) ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\\n' 
LOCATION '{}' 
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_2_name, s3_private_path_quitline
)
print(statement_quitline)


CREATE EXTERNAL TABLE IF NOT EXISTS cardiovale_db.quitline_fixed(
         Year INT,
         Date INT,
         Date_Ref STRING,
         LocationAbbr STRING,
         LocationDesc STRING,
         TopicType STRING,
         TopicDesc STRING,
         MeasureDesc STRING,
         Sub_Measure STRING,
         Variable STRING,
         Offered_for_Free STRING,
         Offered_for_Free_Text STRING,
         Geographic_Requirements STRING,
         Geographic_Requirements_Text STRING,
         Age_Requirements STRING,
         Age_Requirements_Text STRING,
         Readiness_to_Quit_Requirements STRING,
         Readiness_to_Quit_Requirements_Text STRING,
         Counseling_Requirements STRING,
         Counseling_Requirements_Text STRING,
         Physician_Approval_Required STRING,
         Physician_Approval_Required_Text STRING,
         Medical_Restrictions STRING,
         Medical_Restrictions_Text STRING,
         Insurance_Requirements STRING,
         Insurance_Requirements_Tex

In [103]:
import pandas as pd

pd.read_sql(statement_cardio, conn)

/tmp/ipykernel_2030/1048700737.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_cardio, conn)


""


In [85]:
!aws s3 ls s3://cardiovale-solutions-datascience-pipeline/raw-data/quitline_services/


2025-03-18 02:45:53    6934258 Quitline___Services_Available___Medications_-_2010_To_Present_20250306.csv
2025-03-18 05:07:32    7729582 quitline_fixed.csv


In [27]:
pd.read_sql(statement_quitline, conn)

/tmp/ipykernel_2030/856595348.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_quitline, conn)


""


# Verify The Table Has Been Created Succesfully

In [104]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2030/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,_ipynb_checkpoints
1,cardio_train_cleaned
2,cardio_train_csv
3,quitline___services_available___medications___...
4,quitline_fixed


In [87]:
if (table_1_name and table_2_name) in df_show.values:
    ingest_create_athena_table_cardio_passed = True

# Run A Sample Query for cardio table

In [105]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, "cardio_train_cleaned"
)

print(statement)


SELECT * FROM cardiovale_db.cardio_train_cleaned LIMIT 10


In [106]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_2030/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [62]:
import awswrangler as wr

df = wr.s3.read_csv("s3://cardiovale-solutions-datascience-pipeline/raw-data/cardio_train/cardio_train.csv")
print(df.head())


[03/18/25 04:55:14] INFO     Initializing a Ray instance                                               ]8;id=259738;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py\_core.py]8;;\:]8;id=683368;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py#202\202]8;;\

2025-03-18 04:55:15,974	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1895804928 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.73gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-03-18 04:55:16,138	INFO worker.py:1786 -- Started a local Ray instance.


  id;age;gender;height;weight;ap_hi;ap_lo;cholesterol;gluc;smoke;alco;active;cardio
0              0;18393;2;168;62.0;110;80;1;1;0;0;1;0                               
1              1;20228;1;156;85.0;140;90;3;1;0;0;1;1                               
2              2;18857;1;165;64.0;130;70;3;1;0;0;0;1                               
3             3;17623;2;169;82.0;150;100;1;1;0;0;1;1                               
4              4;17474;1;156;56.0;100;60;1;1;0;0;0;0                               


# Run A Sample Query for quitline table

In [32]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, "quitline_fixed"
)

print(statement)


SELECT * FROM cardiovale_db.quitline_fixed LIMIT 10


In [33]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_2030/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,year,date,date_ref,locationabbr,locationdesc,topictype,topicdesc,measuredesc,sub_measure,variable,...,number_of_weeks_offered,limit_per_year,comments,geolocation,topictypeid,topicid,measureid,source,submeasureid,displayorder
0,2020,12,Jul-Dec,GA,Georgia,Quitline,Services Available,Medications,Nicotine Lozenge,nan,...,NaN,nan,nan,"""(32.83968109300048","-83.62758034599966)""",QUI,900QUI,912MED,"""National Quitline Data Warehouse",None
1,2020,12,Jul-Dec,MO,Missouri,Quitline,Services Available,Medications,Nicotine Lozenge,nan,...,NaN,nan,nan,"""(38.635790776000476","-92.56630005299968)""",QUI,900QUI,912MED,"""National Quitline Data Warehouse",None
2,2020,12,Jul-Dec,MT,Montana,Quitline,Services Available,Medications,Bupropion (Zyban®),nan,...,NaN,nan,nan,"""(47.06652897200047","-109.42442064499971)""",QUI,900QUI,912MED,"""National Quitline Data Warehouse",None
3,2020,12,Jul-Dec,NJ,New Jersey,Quitline,Services Available,Medications,Nicotine Lozenge,nan,...,NaN,nan,nan,"""(40.13057004800049","-74.27369128799967)""",QUI,900QUI,912MED,"""National Quitline Data Warehouse",None
4,2020,12,Jul-Dec,OK,Oklahoma,Quitline,Services Available,Medications,Nicotine Gum,All Eligible Callers,...,2.0,2 times per year,nan,"""(35.47203135600046","-97.52107021399968)""",QUI,900QUI,912MED,"""National Quitline Data Warehouse",None


In [27]:
import awswrangler as wr

df = wr.s3.read_csv("s3://cardiovale-solutions-datascience-pipeline/raw-data/Quitline___Services_Available___Medications_-_2010_To_Present_20250306.csv")
print(df.head())


   Year  Date Date_Ref LocationAbbr LocationDesc TopicType  \
0  2020    12  Jul-Dec           GA      Georgia  Quitline   
1  2020    12  Jul-Dec           MO     Missouri  Quitline   
2  2020    12  Jul-Dec           MT      Montana  Quitline   
3  2020    12  Jul-Dec           NJ   New Jersey  Quitline   
4  2020    12  Jul-Dec           OK     Oklahoma  Quitline   

            TopicDesc  MeasureDesc         Sub-Measure              Variable  \
0  Services Available  Medications    Nicotine Lozenge                   NaN   
1  Services Available  Medications    Nicotine Lozenge                   NaN   
2  Services Available  Medications  Bupropion (Zyban®)                   NaN   
3  Services Available  Medications    Nicotine Lozenge                   NaN   
4  Services Available  Medications        Nicotine Gum  All Eligible Callers   

   ... Number_of_Weeks_Offered    Limit_Per_Year Comments  \
0  ...                     NaN               NaN      NaN   
1  ...                  

*** SIGTERM received at time=1742177708 on cpu 0 ***
PC: @     0x7f3866128e2e  (unknown)  epoll_wait
    @     0x7f37fe1a64fd         64  absl::lts_20230802::AbslFailureSignalHandler()
    @     0x7f3866045520  (unknown)  (unknown)
[2025-03-17 02:15:08,662 E 127 127] logging.cc:440: *** SIGTERM received at time=1742177708 on cpu 0 ***
[2025-03-17 02:15:08,662 E 127 127] logging.cc:440: PC: @     0x7f3866128e2e  (unknown)  epoll_wait
[2025-03-17 02:15:08,665 E 127 127] logging.cc:440:     @     0x7f37fe1a6529         64  absl::lts_20230802::AbslFailureSignalHandler()
[2025-03-17 02:15:08,665 E 127 127] logging.cc:440:     @     0x7f3866045520  (unknown)  (unknown)


In [21]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [67]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

/tmp/ipykernel_1040/4130537117.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Store Variables for the Next Notebooks

In [23]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_tsv_passed             -> True
s3_private_path_tsv                               -> 's3://sagemaker-us-east-1-786782285170/amazon-revi
s3_public_path_tsv                                -> 's3://usd-mads-508/amazon-reviews-pds/tsv'
setup_dependencies_passed                         -> True
setup_iam_roles_passed                            -> True
setup_instance_check_passed                       -> True
setup_s3_bucket_passed                            -> True


# Release Resources

In [24]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}